In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import scipy.optimize as spo

In [1]:
%matplotlib notebook

In [4]:
# Sample thickness
t = 3*10**-6

In [7]:
# Sheet resistance measurements
V1 = np.array([21.68, 21.68, 21.68, 21.68,
              6.297, 6.296, 6.295, 6.297]) * 10**-3
# Hall resistance measurements
V2 = np.array([15.389, 15.388, -15.393, -15.393]) * 10**-3
I = 3 * 10**-3

# Lab data
----------------------------------------------------------------------

## n-type GaAs (t=3$\mu$m):

#### Measuring the sheet resistance, $R_{sh}$

| Current I, (mA) |Voltage, $V_{ij,kl}$| Value, (mV) | Resistance ($\Omega$)
| --------------- |:------------:|:------------:|:------------:|
|       3         | $V_{14,23}$  | 21.68 | 7.23
|       3         | $V_{41,32}$  | 21.68 | 7.23
|       3         | $V_{23,14}$  | 21.68 | 7.23
|       3         | $V_{32,41}$  | 21.68 | 7.23
|       3         | $V_{12,43}$  | 6.297 | 2.10
|       3         | $V_{21,34}$  | 6.296 | 2.10
|       3         | $V_{43,12}$  | 6.295 | 2.10
|       3         | $V_{34,21}$  | 6.297 | 2.10
|       3         | $V_{42,31}$  | 15.393| 

#### Measuring the Hall Resistance
| Current I, (mA) |Voltage, $V_{ij,kl}$| Value, (mV) | Resistance ($\Omega$)
| --------------- |:------------:|:------------:|:------------:|
|       3         | $V_{13,24}$  | 15.489 | 5.13
|       3         | $V_{31,42}$  | 15.488 | 5.13
|       3         | $V_{24,31}$  | -15.493 | 5.13
|       3         | $V_{42,13}$  | -15.493 | 5.13

In [8]:
Ra = np.average(R1[4:])
Rb = np.average(R1[0:4])
ratio = 21.68/15.393

In [12]:
def f(y, x=1.408):
    return np.cosh((x-1)/(x+1) * np.log(2)/y) - 0.5*np.exp(np.log(2)/y)

In [10]:
x0 = ratio
z = spo.brentq(f, 0.8, 0.99, (x0))

In [13]:
# Sheet resistance
R_sh = (np.pi / np.log(2)) * ((Ra+Rb)/2) * z
# Resistivity
rho = R_sh * t
print 'rho = ' + str(rho)+' Ohm m'

rho = 6.27623992901e-05 Ohm m


#### $R_{H}$ : Measuring the Hall coefficient, for fixed magnetic field strength

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 100.2   | 1.526
|       2.70         | 100.2   | 1.374
|       2.40         | 100.2   | 1.221
|       2.10         | 100.2   | 1.068
|       2.00         | 100.2   | 1.017
|       1.80         | 100.2   | 0.915
|       1.50         | 100.2   | 0.763
|       1.20         | 100.2   | 0.610
|       0.90         | 100.2   |0.510
|       0.60         | 100.2   |0.306
|       0.30         | 100.2   |0.153



Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 100.2   | -1.525
|       2.70         | 100.2   | -1.373
|       2.40         | 100.2   | -1.221
|       2.10         | 100.2   | -1.068
|       1.80         | 100.2   | -0.915
|       1.50         | 100.2   | -0.732
|       1.20         | 100.2   | -0.611
|       0.90         | 100.2   | -0.458
|       0.60         | 100.2   | -0.305
|       0.30         | 100.2   | -0.153

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -100.3   | -1.438
|       2.70         | -100.3   | -1.294
|       2.40         | -100.3   | -1.151
|       2.10         | -100.3   | -1.008
|       1.80         | -100.3   | -0.864
|       1.50         | -100.3   | -0.720
|       1.20         | -100.3   | -0.577
|       0.90         | -100.3   | -0.433
|       0.60         | -100.3   | -0.289
|       0.30         | -100.3   | -0.145

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -100.3   | 1.437
|       2.70         | -100.3   | 1.292
|       2.40         | -100.3   | 1.149
|       2.10         | -100.3   | 1.006
|       1.80         | -100.3   | 0.862
|       1.50         | -100.3   | 0.718
|       1.20         | -100.3   | 0.575
|       0.90         | -100.3   | 0.431
|       0.60         | -100.3   | 0.288
|       0.30         | -100.3   | 0.144

-------------------------------------------


Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 150.1   | 2.407
|       2.70         | 150.1   | 2.166
|       2.40         | 150.1   | 1.926
|       2.10         | 150.2   | 1.686
|       1.80         | 150.2   | 1.445
|       1.50         | 150.3   | 1.205
|       1.20         | 150.3   | 0.965
|       0.90         | 150.3   |0.723
|       0.60         | 150.4   |0.483
|       0.30         | 149.8   |0.242

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 150.1   | -2.406
|       2.70         | 150.1   | -2.166
|       2.40         | 150.1   | -1.926
|       2.10         | 150.1   | -1.685
|       1.80         | 150.1   | -1.445
|       1.50         | 150.1   | -1.204
|       1.20         | 150.1   | -0.963
|       0.90         | 150.1   | -0.723
|       0.60         | 150.0   | -0.482
|       0.30         | 150.1   | -0.242

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -150.4   | 2.415
|       2.70         | -150.4   | 2.175
|       2.40         | -150.4   | 1.934
|       2.10         | -150.3   | 1.691
|       1.80         | -150.4   | 1.449
|       1.50         | -150.3   | 1.208
|       1.20         | -150.3   | 0.967
|       0.90         | -150.3   | 0.725
|       0.60         | -150.2   | 0.482
|       0.30         | -150.1   | 0.243

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -150.5   | -2.415
|       2.70         | -150.8   | -2.173
|       2.40         | -150.8   | -1.932
|       2.10         | -150.8   | -1.691
|       1.80         | -150.8   | -1.449
|       1.50         | -150.8   | -1.208
|       1.20         | -150.8   | -0.967
|       0.90         | -150.8   | -0.725
|       0.60         | -150.8   | -0.484
|       0.30         | -150.8   | -0.243


-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 204.0   | 3.264
|       2.70         | 203.1   | 2.937
|       2.40         | 203.1   | 2.611
|       2.10         | 203.1   | 2.285
|       1.80         | 203.1   | 1.959
|       1.50         | 203.1   | 1.633
|       1.20         | 203.1   | 1.306
|       0.90         | 203.1   | 0.981
|       0.60         | 203.1   | 0.654
|       0.30         | 203.1   | 0.326

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 203.1   | -3.263
|       2.70         | 203.1   | -2.937
|       2.40         | 203.1   | - 2.612 
|       2.10         | 203.1   | - 2.285
|       1.80         | 203.1   | -1.959
|       1.50         | 203.1   | -1.633
|       1.20         | 203.1   | -1.306
|       0.90         | 203.1   | -0.980
|       0.60         | 203.1   | -0.654
|       0.30         | 203.1       | -0.328

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -200.7   | 2.672
|       2.70         | -200.5   | 2.404
|       2.40         | -200.5   | 2.135
|       2.10         | -200.3   | 1.867
|       1.80         | -200.3   | 1.599
|       1.50         | -200.0   | 1.333
|       1.20         | -199.8   | 1.065
|       0.90         | -199.7   | 0.798
|       0.60         | -199.5   | 0.533
|       0.30         | -200.0   | 0

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -201.9   | -2.689
|       2.70         | -201.7   | -2.417
|       2.40         | -201.6   | -2.147
|       2.10         | -201.5   | -1.878
|       1.80         | -201.4   | -1.609
|       1.50         | -201.3   | -1.340
|       1.20         | -201.2   | -1.072
|       0.90         | -201.1   | -0.803
|       0.60         | -200.9   | -0.536
|       0.30         | -200.8   | -0.268


-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 248.9   | 3.314
|       2.70         | 248.7   | 2.981
|       2.40         | 248.6   | 2.648
|       2.10         | 248.6   | 2.317
|       1.80         | 249.5   | 1.985
|       1.50         | 248.4   | 1.653
|       1.20         | 249.3   | 1.322
|       0.90         | 249.3   |0.992
|       0.60         | 249.3   |0.662
|       0.30         | 249.0   |0.332

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 249.9   | -3.329
|       2.70         | 249.8   | -2.995
|       2.40         | 249.7   | -2.661
|       2.10         | 249.6   | -2.327
|       1.80         | 249.5   | -1.994
|       1.50         | 249.4   | -1.661
|       1.20         | 249.3   | -1.329
|       0.90         | 249.2   | -0.996
|       0.60         | 249.1   | -0.664
|       0.30         | 249.9   | -0.333

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -249.9   | 3.319
|       2.70         | -249.9   | 2.986
|       2.40         | -249.7   | 2.653
|       2.10         | -249.6   | 2.321
|       1.80         | -249.5   | 1.989
|       1.50         | -249.4   | 1.658
|       1.20         | -248.3   | 1.326
|       0.90         | -249.2   | 0.994
|       0.60         | -248.1   | 0.663
|       0.30         | -249.1   | 0.349

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -250.3   | -3.329
|       2.70         | -249.7   | -2.995
|       2.40         | -249.6   | -2.661
|       2.10         | -249.5   | -2.329
|       1.80         | -249.4   | -1.995
|       1.50         | -249.3   | -1.663
|       1.20         | -249.3   | -1.328
|       0.90         | -249.2   | -0.998
|       0.60         | -249.1   | -0.665
|       0.30         | -249.0   | -0.333



-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 301   | 4.046
|       2.70         | 301   | 3.641
|       2.40         | 301   | 3.235
|       2.10         | 301 | 2.831
|       1.80         | 301   | 2.427
|       1.50         | 301   | 2.022
|       1.20         | 301     | 1.618
|       0.90         | 301   | 1.213
|       0.60         | 301   | 0.809
|       0.30         | 301   | 0.406

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 302   | -4.049
|       2.70         | 302   | -3.644
|       2.40         | 302   | -3.240
|       2.10         | 302   | -2.834
|       1.80         | 302   | -2.430
|       1.50         | 302   | -2.024
|       1.20         | 302   | -1.619
|       0.90         | 302   | -1.215
|       0.60         | 301   | -0.810
|       0.30         | 301   | -0.405

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -301   | 4.017
|       2.70         | -301   | 3.615
|       2.40         | -301   | 3.213
|       2.10         | -301   | 2.811
|       1.80         | -301   | 2.410
|       1.50         | -301   | 2.008
|       1.20         | -301   | 1.607
|       0.90         | -301   | 1.205
|       0.60         | -301   | 0.804
|       0.30         | -301   | 0.403

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -301   | -4.020
|       2.70         | -301   | -3.617
|       2.40         | -301   | -3.215
|       2.10         | -301   | -2.814
|       1.80         | -301   | -2.411
|       1.50         | -301   | -2.010
|       1.20         | -301   | -1.608
|       0.90         | -301   | -1.206
|       0.60         | -301   | -0.804
|       0.30         | -301   | -0.405

In [14]:
I = np.arange(0.3,3.3,0.3)*10**-3
V1 = np.array([0.153, 0.306, 0.510, 0.610, 0.763, 0.915, 1.068, 1.221, 1.374, 1.526]) * 10**-3
V2 = np.array([-0.153, -0.305, -0.458, -0.611, -0.732, -0.915, -1.068, -1.221, -1.373, -1.525]) * 10**-3
V4 = np.array([-0.145, -0.289, -0.433, -0.577, -0.720, -0.846, -1.008, -1.151, -1.294, -1.438]) * 10**-3
V3 = np.array([0.144, 0.288, 0.431, 0.571, 0.718, 0.862, 1.006, 1.149, 1.292, 1.437]) * 10**-3
V = (V1 - V2 + V3 - V4)/4
B = 102.5 * 10**-3
IB = I*B
print V

[ 0.00014875  0.000297    0.000458    0.00059225  0.00073325  0.0008845
  0.0010375   0.0011855   0.00133325  0.0014815 ]


In [10]:
fig1 = plt.figure()
ax = fig1.add_subplot(111)
ax.set_title('A plot of Current against Voltage, at fixed magnetic field strength')
#ax.legend('For fixed B=0.374mT')
ax.plot(IB, V)
print V

FigureCanvasNbAgg()

[ 0.00014875  0.000297    0.000458    0.00059225  0.00073325  0.0008845
  0.0010375   0.0011855   0.00133325  0.0014815 ]


In [11]:
def y(x, A, B): # this is your 'straight line' y=f(x)
    return A*x + B
R_h_eff,foo = curve_fit(y, IB, V)[0] # your data x, y to fit

In [12]:
print 'R_h effective= ' + str(R_h_eff) + ' Ohm/Tesla'

R_h effective= 4.8037447671 Ohm/Tesla


In [13]:
mu = R_h_eff/rho * 3*10**-6
print 'mu = ' + str(mu) + ' m^2/Vs'
n = (R_h_eff*3*10**-6*1.67*10**-19)**-1
print 'n = ' + str(n) + ' m^-3'

mu = 0.229615732737 m^2/Vs
n = 4.1551083182e+23 m^-3


$\mu = 2296 {cm}^{2}V^{-1}s^{-1}$

$n = 4.155x10^{17} {cm}^{-3}$

In [15]:
B

0.10250000000000001